## Model 1.1 GC

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

''

## Import the necessary libraries and set up the path to your data

In [2]:
import os
import json
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Path to the folder containing your data
data_dir = 'C:/Users/8saks/Documents/GitHub/Flower-Recognition-AI-ML-Model/Combined flowers'

## Initialize ImageDataGenerator with augmentation

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

## Prepare the training and validation generators

In [4]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    data_dir, 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 2505 images belonging to 10 classes.
Found 621 images belonging to 10 classes.


## Load the EfficientNetB7 model and define your own model

In [5]:
base_model = EfficientNetB7(weights='imagenet', include_top=False)

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes, activation='softmax'))

258076736/258076736 [==============================] - 71s 0us/step


##  Compile the model

In [6]:
model.compile(optimizer=Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

## Define the early stop and reduce learning rate callbacks

In [7]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

## Train the model

In [ ]:
history = model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50,  
    callbacks=[early_stop, reduce_lr])

Epoch 1/50


## Evaluate the model on validation data

In [ ]:
loss, accuracy = model.evaluate(validation_generator)

print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

## Generate and print the classification report and confusion matrix

In [ ]:
y_true = validation_generator.classes
predictions = model.predict(validation_generator)
y_pred = np.argmax(predictions, axis=1)

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

## Save the model and the label to index mapping

In [ ]:
# Save the model and the label to index mapping
model_path = 'C:/Users/8saks/Documents/GitHub/Flower-Recognition-AI-ML-Model/flower_classification_model.h5'
model.save(model_path)

label2index = train_generator.class_indices
label2index_path = 'C:/Users/8saks/Documents/GitHub/Flower-Recognition-AI-ML-Model/label2index.json'

with open(label2index_path, 'w') as f:
    json.dump(label2index, f)

## Accuracy and loss curves

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(14,5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## Displaying actual and predicted correct/incorrect flowers:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get a batch of data
x, y_true = next(validation_generator)

# Predict on the batch of data
y_pred = model.predict(x)
y_pred = np.argmax(y_pred, axis=1)

# Get the 'true' label names
label_map = validation_generator.class_indices
label_map = {v:k for k, v in label_map.items()}  # flip k,v to v,k

# Display a few correctly classified images
correct_indices = np.where(y_pred == y_true)[0]
if len(correct_indices) > 0:
    plt.figure(figsize=(10, 10))
    for i, correct_index in enumerate(correct_indices[:9]):
        plt.subplot(3, 3, i + 1)
        image = x[correct_index]
        plt.imshow(image)
        plt.title(f"Predicted: {label_map[y_pred[correct_index]]} \nActual: {label_map[y_true[correct_index]]}")
        plt.axis('off')

## Classification report and confusion matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = validation_generator.classes
predictions = model.predict(validation_generator)
y_pred = np.argmax(predictions, axis=1)

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))